In [1]:
import os
import openai
import glob
import shutil
openai.api_key = os.getenv("OPENAI_API_KEY")

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests
import re

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import base64
import email
from email import policy
from email.parser import BytesParser
from email.mime.text import MIMEText

from bs4 import BeautifulSoup
import dateutil.parser as parser

import sys
sys.path.insert(0, './functions/untested functions/')
sys.path.insert(0, './functions/tested functions/')
from gptLearning import *
import gptLearning as gl

In [2]:
def retrieve_emails(n=1, user_id='me'):
    """
    获取指定数量的最近邮件。

    参数:
    n: 要检索的邮件的数量。这应该是一个整数。
    user_id: 要检索邮件的用户的ID。默认值是'me'，表示当前授权的用户。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'From': 发件人的邮箱地址。
    'Date': 邮件的发送日期。
    'Subject': 邮件的主题。
    'Snippet': 邮件的摘要（前100个字符）。
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')

    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)

    # 获取邮件列表
    results = service.users().messages().list(userId=user_id).execute()
    messages = results.get('messages', [])[:n]

    emails = []
    for message in messages:
        msg = service.users().messages().get(userId=user_id, id=message['id']).execute()

        # 解码邮件内容
        payload = msg['payload']
        headers = payload.get("headers")
        parts = payload.get("parts")

        data = {}

        if headers:
            for d in headers:
                name = d.get("name")
                if name.lower() == "from":
                    data['From'] = d.get("value")
                if name.lower() == "date":
                    data['Date'] = parser.parse(d.get("value")).strftime('%Y-%m-%d %H:%M:%S')
                if name.lower() == "subject":
                    data['Subject'] = d.get("value")

        if parts:
            for part in parts:
                mimeType = part.get("mimeType")
                body = part.get("body")
                data_decoded = base64.urlsafe_b64decode(body.get("data")).decode()
                if mimeType == "text/plain":
                    data['Snippet'] = data_decoded
                elif mimeType == "text/html":
                    soup = BeautifulSoup(data_decoded, "html.parser")
                    data['Snippet'] = soup.get_text()

        emails.append(data)

    # 返回邮件列表
    return json.dumps(emails, indent=2)

In [3]:
functions_list = [get_latest_email,send_email,retrieve_emails,get_email_count]

In [4]:
functions = auto_functions(functions_list)

In [5]:
functions

[{'name': 'get_latest_email',
  'description': '查询Gmail邮箱中最后一封邮件信息',
  'parameters': {'type': 'object',
   'properties': {'userId': {'type': 'string',
     'description': "需要查询的邮箱ID，当查询我的邮箱时，userId需要输入'me'"}},
   'required': ['userId']}},
 {'name': 'send_email',
  'description': '借助Gmail API创建并发送邮件函数',
  'parameters': {'type': 'object',
   'properties': {'to': {'type': 'string', 'description': '邮件发送的目标邮箱地址'},
    'subject': {'type': 'string', 'description': '邮件主题'},
    'message_text': {'type': 'string', 'description': '邮件全部正文'}},
   'required': ['to', 'subject', 'message_text']},
  'return': {'type': 'object',
   'properties': {'mail_id': {'type': 'string', 'description': '发送成功后的邮件ID'},
    'status': {'type': 'string', 'description': '发送状态'}}}},
 {'name': 'retrieve_emails',
  'description': '获取指定数量的最近邮件',
  'parameters': {'type': 'object',
   'properties': {'n': {'type': 'integer', 'description': '要检索的邮件的数量'},
    'user_id': {'type': 'string',
     'default': 'me',
     'description':

In [6]:
messages = [
    {'role':"user","content":"请查看我的邮箱里是否有黄剑的邮件，如果有的话请帮解读下内容。"}
]

In [7]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions,
        function_call="auto",  
    )

In [8]:
response.choices[0]['message']

<OpenAIObject at 0x15ba92e80> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "retrieve_emails",
    "arguments": "{\n  \"user_id\": \"me\"\n}"
  }
}

In [ ]:
chat_with_model(prompt='你好',functions_list=functions_list)

In [9]:
get_email_input = "请帮我查下邮箱里最后一封邮件内容。"

get_email_out = "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：\
                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                 3.请将全部功能封装在一个函数内；\
                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

email_counts_input = "请帮我查下邮箱里现在总共有多少封邮件。"

email_counts_out = "请帮我编写一个python函数，用于查看我的Gmail邮箱中总共有多少封邮件，函数要求如下：\
                    1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                    2.函数返回结果是当前邮件总数，返回结果本身必须是一个json格式对象；\
                    3.请将全部功能封装在一个函数内；\
                    4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [10]:
user_content = "请查下我的邮箱里是否有来自黄剑的未读邮件，并解读最近一封未读邮件的内容"

messages_fewShot_stage1 = [{"role": "system", "content": "请按照格式，编写一段函数功能说明。"},
                          {"role": "user", "name":"example1_user", "content": get_email_input},
                          {"role": "assistant", "name":"example1_assistant", "content": get_email_out},
                          {"role": "user", "name":"example2_user", "content": email_counts_input},
                          {"role": "assistant", "name":"example2_assistant", "content": email_counts_out},
                          {"role": "user", "name":"example_user", "content": user_content}]
messages_fewShot_stage1

[{'role': 'system', 'content': '请按照格式，编写一段函数功能说明。'},
 {'role': 'user', 'name': 'example1_user', 'content': '请帮我查下邮箱里最后一封邮件内容。'},
 {'role': 'assistant',
  'name': 'example1_assistant',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'user', 'name': 'example2_user', 'content': '请帮我查下邮箱里现在总共有多少封邮件。'},
 {'role': 'assistant',
  'name': 'example2_assistant',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中总共有多少封邮件，函数要求如下：                    1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                    2.函数返回结果是当前邮件总数，返回结果本身必须是一个json格式对象；                    3.请将全部功能封装在一个函数内；                    4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'user',
  'name': 'example_user',
  'content': '请查下我的邮箱里是否有来自黄剑的

In [11]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages=messages_fewShot_stage1
)

In [12]:
print(response.choices[0].message['content'])

请帮我编写一个python函数，用于查看我的Gmail邮箱中是否有来自黄剑的未读邮件，并且解读最近一封未读邮件的内容，函数要求如下：
1. 函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；
2. 函数返回结果是一个字典对象，包含两个字段：
   - "has_unread_email": 一个布尔值，表示是否有来自黄剑的未读邮件；
   - "latest_unread_email": 如果有未读邮件，则表示最近一封未读邮件的内容，格式为一个字典对象，包含邮件的发送者、主题、日期和内容等信息；
3. 请将全部功能封装在一个函数内；
4. 请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；


In [13]:
system_content = "我现在已完成Gmail API授权，授权文件为本地文件token.json。"

In [14]:
user_example_content = "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：\
                        1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                        2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                        3.请将全部功能封装在一个函数内；\
                        4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [15]:
with open('./functions/tested functions/%s_module.py' % 'get_latest_email', encoding='utf-8') as f:
    assistant_example_content = f.read()

In [16]:
assistant_example_content

'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        \n    return json.dumps(msg)\n'

In [17]:
user_content = response.choices[0].message['content']

In [18]:
user_content

'请帮我编写一个python函数，用于查看我的Gmail邮箱中是否有来自黄剑的未读邮件，并且解读最近一封未读邮件的内容，函数要求如下：\n1. 函数参数userId，userId是字符串参数，默认情况下取值为\'me\'，表示查看我的邮件；\n2. 函数返回结果是一个字典对象，包含两个字段：\n   - "has_unread_email": 一个布尔值，表示是否有来自黄剑的未读邮件；\n   - "latest_unread_email": 如果有未读邮件，则表示最近一封未读邮件的内容，格式为一个字典对象，包含邮件的发送者、主题、日期和内容等信息；\n3. 请将全部功能封装在一个函数内；\n4. 请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；'

In [19]:
messages_fewShot_stage2 = [{"role": "system", "content": system_content},
                          {"role": "user", "name":"example_user", "content": user_example_content},
                          {"role": "assistant", "name":"example_assistant", "content": assistant_example_content},
                          {"role": "user", "name":"example_user", "content": user_content}]

In [20]:
response2 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages=messages_fewShot_stage2
)

In [21]:
s = response2.choices[0]['message']['content']

In [22]:
extract_function_code(s,detail=1)

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

def check_unread_email(userId='me'):
    """
    查询Gmail邮箱中是否有来自黄剑的未读邮件，并解读最近一封未读邮件的内容
    :param userId: 邮箱ID，默认为'me'，表示查看我的邮件
    :return: 包含两个字段的字典对象：
             - "has_unread_email": 一个布尔值，表示是否有来自黄剑的未读邮件
             - "latest_unread_email": 如果有未读邮件，则表示最近一封未读邮件的内容，格式为一个字典对象，
                                      包含邮件的发送者、主题、日期和内容等信息
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')
    
    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)
    
    # 搜索未读邮件来源与黄剑的邮件
    query = "is:unread from:黄剑"
    results = service.users().messages().list(userId=userId, q=query).execute()
    messages = results.get('messages', [])
    
    has_unread_email = len(messages) > 0
    latest_unread_email = {}
    
    if has_unread_email:
        # 获取最近一封未读邮件的详细信息
        message = messages[0]
        msg = service.users().messages().get(userId

In [37]:
from check_unread_email_module import check_unread_email

In [38]:
check_unread_email()

{'has_unread_email': False, 'latest_unread_email': {}}

In [35]:
del check_unread_email

In [39]:
system_content1 = "为了更好编写满足用户需求的python函数，我们需要先识别用户需求中的变量，以作为python函数的参数。需要注意的是，当前编写的函数中涉及到的邮件收发查阅等功能，都是通过调用Gmail API来完成。"

In [40]:
input1 = "请帮我查下Gmail邮箱里最后一封邮件内容。"
pi1 = "当前需求中可以作为函数参数的是：1.查看谁的邮箱。"

In [52]:
input2 = "请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。"
pi2 = "当前需求中可以作为函数参数的是：1.谁发送的邮件；2.发送邮件的主题；3.邮件具体内容"

In [63]:
input3 = "请查下我的邮箱，看看是否有阿新发送来的未读邮件，并解读最近一封未读邮件的内容。"

In [64]:
messages_CD = [
    {"role": "system", "content": system_content1},
    {"role": "user", "name": "example1_user", "content": input1},
    {"role": "assistant", "name": "example1_assistant", "content": pi1},
    {"role": "user", "name": "example2_user", "content": input2},
    {"role": "assistant", "name": "example2_assistant", "content": pi2}, 
    {"role": "user", "name": "example_user", "content": input3}
]

In [65]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages=messages_CD
)

In [66]:
pi3 = response.choices[0].message['content']

In [67]:
pi3

'当前需求中可以作为函数参数的是：1.查看谁的邮箱；2.筛选未读邮件的发送者。'

In [68]:
system_content2 = "我现在已完成Gmail API授权，授权文件为本地文件token.json。所编写的函数要求参数类型都必须是字符串类型。"

In [69]:
get_email_input = "请帮我查下邮箱里最后一封邮件内容。" + pi1
get_email_input

'请帮我查下邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'

In [70]:
get_email_out = "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：\
                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                 3.请将全部功能封装在一个函数内；\
                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [72]:
send_email_input = "请帮我给阿新发送一封Gmail邮件，他的邮箱是ax909@163.com,请他明天早上9点半来我办公室开会，商量下半年技术开发计划。" + pi2
send_email_input

'请帮我给阿新发送一封Gmail邮件，他的邮箱是ax909@163.com,请他明天早上9点半来我办公室开会，商量下半年技术开发计划。当前需求中可以作为函数参数的是：1.谁发送的邮件；2.发送邮件的主题；3.邮件具体内容'

In [73]:
send_email_out = "请帮我编写一个python函数，用于给阿新发送邮件，ax909@163.com，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：\
                  1.函数参数to、subject和message_text，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，message_text表示邮件具体内容；\
                  2.函数返回结果是当前邮件发送状态，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [74]:
user_content = input3 + pi3
user_content

'请查下我的邮箱，看看是否有阿新发送来的未读邮件，并解读最近一封未读邮件的内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱；2.筛选未读邮件的发送者。'

In [75]:
messages_CM = [{"role": "system", "content": system_content2},
                   {"role": "user", "name":"example1_user", "content": get_email_input},
                   {"role": "assistant", "name":"example1_assistant", "content": get_email_out},
                   {"role": "user", "name":"example2_user", "content": send_email_input},
                   {"role": "assistant", "name":"example2_assistant", "content": send_email_out},
                   {"role": "user", "name":"example_user", "content": user_content}]

In [76]:
messages_CM 

[{'role': 'system',
  'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。所编写的函数要求参数类型都必须是字符串类型。'},
 {'role': 'user',
  'name': 'example1_user',
  'content': '请帮我查下邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
 {'role': 'assistant',
  'name': 'example1_assistant',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'user',
  'name': 'example2_user',
  'content': '请帮我给阿新发送一封Gmail邮件，他的邮箱是ax909@163.com,请他明天早上9点半来我办公室开会，商量下半年技术开发计划。当前需求中可以作为函数参数的是：1.谁发送的邮件；2.发送邮件的主题；3.邮件具体内容'},
 {'role': 'assistant',
  'name': 'example2_assistant',
  'content': '请帮我编写一个python函数，用于给阿新发送邮件，ax909@163.com，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：                  1.函数参数to、subject和message_text，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，message_text表示邮件具体内容；              

In [81]:
response_CM = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages=messages_CM
)

In [82]:
function_description = response_CM.choices[0].message['content']
function_description

"请帮我编写一个python函数，用于查询指定邮箱中的未读邮件并解读最近一封未读邮件的内容，函数要求如下：             \n                1.函数参数userId和sender两个参数，都是字符串类型。userId表示要查询的邮箱用户，sender表示筛选未读邮件的发送者。默认情况下userId取值为'me'，表示查询当前用户的邮箱；如果不指定sender参数，则默认查询所有未读邮件；                    \n                2.函数返回结果是最近一封未读邮件的内容，返回结果本身必须是一个json格式对象，包含邮件的主题、发送者、接收者、时间和内容等信息；       \n                3.请将全部功能封装在一个函数内；             \n                4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [80]:
response_CD

<OpenAIObject chat.completion id=chatcmpl-7oYspGaMiAWn9uQjO0cz0G3HYw8f2 at 0x15f1c24d0> JSON: {
  "id": "chatcmpl-7oYspGaMiAWn9uQjO0cz0G3HYw8f2",
  "object": "chat.completion",
  "created": 1692284847,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u5f53\u524d\u9700\u6c42\u4e2d\u53ef\u4ee5\u4f5c\u4e3a\u51fd\u6570\u53c2\u6570\u7684\u662f\uff1a1.\u67e5\u770b\u8c01\u7684\u90ae\u7bb1\u3002"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 288,
    "completion_tokens": 21,
    "total_tokens": 309
  }
}

In [83]:
func1_description = get_email_out
func1_description

"请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [84]:
with open('./functions/tested functions/%s_module.py' % 'get_latest_email', encoding='utf-8') as f:
    func1_str = f.read()

In [85]:
func1_str

'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        \n    return json.dumps(msg)\n'

In [86]:
func2_description = send_email_out
func2_description

'请帮我编写一个python函数，用于给阿新发送邮件，ax909@163.com，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：                  1.函数参数to、subject和message_text，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，message_text表示邮件具体内容；                  2.函数返回结果是当前邮件发送状态，返回结果本身必须是一个json格式对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；'

In [90]:
s = inspect.getsource(send_email)

In [94]:
extract_function_code(s,detail=1,tested=True)

def send_email(to, subject, message_text):
    """
    借助Gmail API创建并发送邮件函数
    :param to: 必要参数，字符串类型，用于表示邮件发送的目标邮箱地址；
    :param subject: 必要参数，字符串类型，表示邮件主题；
    :param message_text: 必要参数，字符串类型，表示邮件全部正文；
    :return：返回发送结果字典，若成功发送，则返回包含邮件ID和发送状态的字典。
    """
    
    creds_file='token_send.json'
    
    def create_message(to, subject, message_text):
        """创建一个MIME邮件"""
        message = MIMEText(message_text)
        message['to'] = to
        message['from'] = 'me'
        message['subject'] = subject
        raw_message = base64.urlsafe_b64encode(message.as_string().encode('utf-8')).decode('utf-8')
        return {
            'raw': raw_message
        }

    def send_message(service, user_id, message):
        """发送邮件"""
        try:
            sent_message = service.users().messages().send(userId=user_id, body=message).execute()
            print(f'Message Id: {sent_message["id"]}')
            return sent_message
        except Exception as e:
            print(f'An error o

In [95]:
with open('./functions/tested functions/%s_module.py' % 'send_email', encoding='utf-8') as f:
    func2_str = f.read()

In [97]:
func2_str

'def send_email(to, subject, message_text):\n    """\n    借助Gmail API创建并发送邮件函数\n    :param to: 必要参数，字符串类型，用于表示邮件发送的目标邮箱地址；\n    :param subject: 必要参数，字符串类型，表示邮件主题；\n    :param message_text: 必要参数，字符串类型，表示邮件全部正文；\n    :return：返回发送结果字典，若成功发送，则返回包含邮件ID和发送状态的字典。\n    """\n    \n    creds_file=\'token_send.json\'\n    \n    def create_message(to, subject, message_text):\n        """创建一个MIME邮件"""\n        message = MIMEText(message_text)\n        message[\'to\'] = to\n        message[\'from\'] = \'me\'\n        message[\'subject\'] = subject\n        raw_message = base64.urlsafe_b64encode(message.as_string().encode(\'utf-8\')).decode(\'utf-8\')\n        return {\n            \'raw\': raw_message\n        }\n\n    def send_message(service, user_id, message):\n        """发送邮件"""\n        try:\n            sent_message = service.users().messages().send(userId=user_id, body=message).execute()\n            print(f\'Message Id: {sent_message["id"]}\')\n            return sent_message\n        except

In [98]:
system_content3 = "我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。"

In [99]:
messages_stage2 = [{"role": "system", "content": system_content3},
                    {"role": "user", "name":"example1_user", "content": func1_description},
                    {"role": "assistant", "name":"example1_assistant", "content": func1_str},
                    {"role": "user", "name":"example2_user", "content": func2_description},
                    {"role": "assistant", "name":"example2_assistant", "content": func2_str},
                    {"role": "user","content": function_description}]

In [100]:
response_state2 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages=messages_stage2
)

In [101]:
s = response_state2.choices[0].message['content']

In [122]:
s1 = 'from googleapiclient.discovery import build from google.oauth2.credentials import Credentials from google_auth_oauthlib.flow import InstalledAppFlows '+ s

In [103]:
extract_function_code(s1,detail=1,tested=0)

def get_unread_email_content(userId='me', sender=None):
    """
    查询指定邮箱中的未读邮件并解读最近一封未读邮件的内容
    :param userId: 邮箱用户ID，默认为'me'，表示当前用户的邮箱
    :param sender: 筛选未读邮件的发送者，可选参数，默认为None表示查询所有未读邮件
    :return: 最近一封未读邮件的内容，返回结果是一个包含邮件信息的JSON对象
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')
    
    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)
    
    # 查询未读邮件
    query = 'is:unread'
    if sender:
        query += ' from:' + sender
    
    results = service.users().messages().list(userId=userId, q=query).execute()
    messages = results.get('messages', [])
    
    if messages:
        # 获取最近一封未读邮件的详细信息
        msg = service.users().messages().get(userId=userId, id=messages[0]['id']).execute()
        
        # 解读邮件内容
        email_content = {}
        email_content['subject'] = msg.get('subject', '')
        email_content['sender'] = msg.get('from', '')
        email_content['receiver'] = msg.get('to', '')
        emai

In [2]:
from get_unread_email_content_module import get_unread_email_content

In [5]:
functions_list = [get_latest_email, send_email, retrieve_emails, get_email_count]

In [125]:
functions = auto_functions(functions_list)

In [116]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "user", "content": '请查询我的邮箱，看下里面是否有来自阿新的未读邮件。'}
        ],
        functions=functions,
        function_call="auto",  
    )

In [117]:
response.choices[0].message['function_call']['arguments']

'{\n  "sender": "阿新"\n}'

In [8]:
chat_with_model(prompt='你好',functions_list=functions_list)

模型回答: 你好！有什么我可以帮助你的吗？
用户问:请帮我查看是否有阿新发来的邮件？
模型回答: 根据我查找的信息，你收到了一封来自阿新(ax909@163.com)的邮件。这封邮件的主题是"Re:邀请参加会议"，在2023年8月17日上午8:56收到。邮件内容的摘要如下："李沉舟先生，我代表黄剑邀请您于明天早上9点半来我办公室开会，商量下半年技术开发计划。谢谢！黄剑"。邮件是未读状态，标记为重要邮件，并归类到个人分类和收件箱。邮件包括一个纯文本版本和一个HTML版本。
用户问:请帮我查看是否有阿新发来的邮件？
模型回答: 是的，阿新发来了一封邮件。邮件的主题是"Re: 邀请参加会议"，发件人是"阿新"，你想要我将完整邮件内容发送给你吗？
用户问:好的，邮件内容是什么？
模型回答: 阿新发来的邮件的主题是《邀请参加会议》，正文内容如下：

"您好，明天吃早点啦。 在 2023-08-15 22:26:35，jrdnnest@gmail.com 写道： >李沉舟先生， > >>我代表黄剑邀请您于明天早上9点半来我办公室开会，商量下半年技术开发计划。 > >>谢谢！ > 黄剑"
用户问:请回复阿新，明天早餐吃米线。
Message Id: 18a043ae7663844f
模型回答: 邮件已发送给阿新。
用户问:我的邮箱现在有多少邮件？
模型回答: 您的邮箱目前共有201封邮件。
用户问:退出


In [9]:
get_latest_email_CD_input = "请帮我查一下gmail邮箱中最后一封邮件的内容。"
get_latest_email_CD_pi = "当前需求中可以作为函数参数的是：1.查看谁的邮箱。"
get_latest_email_message_CD = [
    {"role": "user", "content": get_latest_email_CD_input},
    {"role": "assistant","content": get_latest_email_CD_pi}
]

In [10]:
get_latest_email_CM_input = get_latest_email_CD_input + get_latest_email_CD_pi
get_latest_email_CM_description = "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件的内容，函数要求如下：\
    1.函数参数userId,userId是字符串参数，默认情况下取值为‘，表示查看我的邮件；\
    2.函数返回结果是一个包含最后一封邮件内容的对象，返回结果必须是一个json格式对象；\
    3.请将全部功能封装在一个函数内；\
    4.请在函数编写的过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况、函数返回结果等信息。；"
get_latest_email_message_CM = [
    {"role": "user", "content": get_latest_email_CM_input},
    {"role": "assistant","content": get_latest_email_CM_description}
]

In [12]:
with open("./functions/tested functions/%s_module.py" % "get_latest_email","r",encoding="utf-8") as f:
    get_latest_email_function = f.read()
get_latest_email_message = [
    {"role": "user", "content": get_latest_email_CM_description},
    {"role": "assistant","content": get_latest_email_function}
]

In [13]:
get_latest_email_prompt = {
    "stage1_CD": get_latest_email_message_CD,
    "stage1_CM": get_latest_email_message_CM,
    "stage2": get_latest_email_message
}

In [14]:
get_latest_email_prompt

{'stage1_CD': [{'role': 'user', 'content': '请帮我查一下gmail邮箱中最后一封邮件的内容。'},
  {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查看谁的邮箱。'}],
 'stage1_CM': [{'role': 'user',
   'content': '请帮我查一下gmail邮箱中最后一封邮件的内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
  {'role': 'assistant',
   'content': '请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件的内容，函数要求如下：    1.函数参数userId,userId是字符串参数，默认情况下取值为‘，表示查看我的邮件；    2.函数返回结果是一个包含最后一封邮件内容的对象，返回结果必须是一个json格式对象；    3.请将全部功能封装在一个函数内；    4.请在函数编写的过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况、函数返回结果等信息。；'}],
 'state2': [{'role': 'user',
   'content': '请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件的内容，函数要求如下：    1.函数参数userId,userId是字符串参数，默认情况下取值为‘，表示查看我的邮件；    2.函数返回结果是一个包含最后一封邮件内容的对象，返回结果必须是一个json格式对象；    3.请将全部功能封装在一个函数内；    4.请在函数编写的过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况、函数返回结果等信息。；'},
  {'role': 'assistant',
   'content': 'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息

In [17]:
with open("./functions/tested functions/%s_prompt.json" % "get_latest_email","w") as f:
    json.dump(get_latest_email_prompt,f)

In [19]:
with open("./functions/tested functions/%s_prompt.json" % "get_latest_email","r") as f:
    data = json.load(f)

In [20]:
data

{'stage1_CD': [{'role': 'user', 'content': '请帮我查一下gmail邮箱中最后一封邮件的内容。'},
  {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查看谁的邮箱。'}],
 'stage1_CM': [{'role': 'user',
   'content': '请帮我查一下gmail邮箱中最后一封邮件的内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
  {'role': 'assistant',
   'content': '请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件的内容，函数要求如下：    1.函数参数userId,userId是字符串参数，默认情况下取值为‘，表示查看我的邮件；    2.函数返回结果是一个包含最后一封邮件内容的对象，返回结果必须是一个json格式对象；    3.请将全部功能封装在一个函数内；    4.请在函数编写的过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况、函数返回结果等信息。；'}],
 'state2': [{'role': 'user',
   'content': '请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件的内容，函数要求如下：    1.函数参数userId,userId是字符串参数，默认情况下取值为‘，表示查看我的邮件；    2.函数返回结果是一个包含最后一封邮件内容的对象，返回结果必须是一个json格式对象；    3.请将全部功能封装在一个函数内；    4.请在函数编写的过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况、函数返回结果等信息。；'},
  {'role': 'assistant',
   'content': 'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息

In [28]:
send_email_CD_input = "请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。"
send_email_pi = "当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容"
send_email_message_CD = [
                          {"role": "user", "content": send_email_CD_input},
                          {"role": "assistant", "content": send_email_pi}
                         ]

send_email_CM_input = send_email_CD_input + send_email_pi
send_email_description = "请帮我编写一个python函数，用于给陈明发送邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：\
                  1.函数参数to、subject和message_text，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，message_text表示邮件具体内容；\
                  2.函数返回结果是当前邮件发送状态，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
send_email_message_CM = [
                          {"role": "user", "content": send_email_CM_input},
                          {"role": "assistant", "content":send_email_description}
                         ]

with open('./functions/tested functions/%s_module.py' % 'send_email', encoding='utf-8') as f:
    send_email_function = f.read()
send_email_message = [
                       {"role": "user", "content": send_email_description},
                       {"role": "assistant", "content":send_email_function}
                      ] 

In [29]:
send_email_prompt = {
                     "stage1_CD": send_email_message_CD,
                     "stage1_CM": send_email_message_CM,
                     "stage2": send_email_message
                    }

with open('./functions/tested functions/%s_prompt.json' % 'send_email', 'w') as f:
    json.dump(send_email_prompt, f)

In [23]:
code_retrieve_emails = inspect.getsource(retrieve_emails)
with open('./functions/tested functions/%s_module.py' % 'retrieve_emails', 'w', encoding='utf-8') as f:
    f.write(code_retrieve_emails)

In [30]:
retrieve_emails_CD_input = "请帮我查看下最近5封邮件的邮件内容"
retrieve_emails_pi = "当前需求中可以作为函数参数的是：1.查询几封邮件；2.查询谁的邮箱"
retrieve_emails_message_CD = [
                          {"role": "user", "content": retrieve_emails_CD_input},
                          {"role": "assistant", "content": retrieve_emails_pi}
                         ]

retrieve_emails_CM_input = retrieve_emails_CD_input + retrieve_emails_pi
retrieve_emails_description = "请帮我编写一个python函数，查询我的邮箱里面最近的几封邮件信息，函数要求如下：\
                  1.函数参数n和user_id，两个参数都是字符串类型，其中n表示查询最近的几封邮件，user_id表示要检索邮件的用户的ID。默认值是'me'，表示当前授权的用户；\
                  2.函数返回结果是查询到的这几封邮件的邮件内容，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
retrieve_emails_message_CM = [
                          {"role": "user", "content": retrieve_emails_CM_input},
                          {"role": "assistant", "content":retrieve_emails_description}
                         ]

with open('./functions/tested functions/%s_module.py' % 'retrieve_emails', encoding='utf-8') as f:
    retrieve_emails_function = f.read()
retrieve_emails_message = [
                       {"role": "user", "content": retrieve_emails_description},
                       {"role": "assistant", "content":retrieve_emails_function}
                      ] 

In [31]:
retrieve_emails_prompt = {
                     "stage1_CD": retrieve_emails_message_CD,
                     "stage1_CM": retrieve_emails_message_CM,
                     "stage2": retrieve_emails_message
                    } 

In [32]:
with open('./functions/tested functions/%s_prompt.json' % 'retrieve_emails', 'w') as f:
    json.dump(retrieve_emails_prompt, f)

In [34]:
get_email_count_CD_input = "请帮我查下邮箱里面总共有多少封邮件。"
get_email_count_pi = "当前需求中可以作为函数参数的是：1.查询谁的邮箱"
get_email_count_message_CD = [
                          {"role": "user", "content": get_email_count_CD_input},
                          {"role": "assistant", "content": get_email_count_pi}
                         ]

get_email_count_CM_input = get_email_count_CD_input + get_email_count_pi
get_email_count_description = "请帮我编写一个python函数，用于查看我的Gmail邮箱中总共有多少封邮件，函数要求如下：\
                  1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                  2.函数返回结果是查询到的这几封邮件的邮件内容，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
get_email_count_message_CM = [
                          {"role": "user", "content": get_email_count_CM_input},
                          {"role": "assistant", "content":get_email_count_description}
                         ]

with open('./functions/tested functions/%s_module.py' % 'get_email_count', encoding='utf-8') as f:
    get_email_count_function = f.read()
get_email_count_message = [
                       {"role": "user", "content": get_email_count_description},
                       {"role": "assistant", "content":get_email_count_function}
                      ] 

In [35]:
get_email_count_prompt = {
                     "stage1_CD": get_email_count_message_CD,
                     "stage1_CM": get_email_count_message_CM,
                     "stage2": get_email_count_message
                    }   

In [36]:
with open('./functions/tested functions/%s_prompt.json' % 'get_email_count', 'w') as f:
    json.dump(get_email_count_prompt, f)

In [37]:
system_messages = {"system_message_CD": [{"role": "system", "content": "为了更好编写满足用户需求的python函数，我们需要先识别用户需求中的变量，以作为python函数的参数。需要注意的是，当前编写的函数中涉及到的邮件收发查阅等功能，都是通过调用Gmail API来完成。"}], 
                   "system_message_CM": [{"role": "system", "content": "我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。"}], 
                   "system_message": [{"role": "system", "content":"我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。"}]}
with open('./functions/tested functions/%s.json' % 'system_messages', 'w') as f:
    json.dump(system_messages, f)

In [38]:
def remove_to_tested(function_name):
    """
    将函数同名文件夹由untested文件夹转移至tested文件夹内。\
    完成转移则说明函数通过测试，可以使用。此时需要将该函数的源码写入gptLearning.py中方便下次调用。
    """
    
    # 将函数代码写入gptLearning.py文件中
    with open('./autoGmail_project/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        function_code = f.read()
    
    with open('gptLearning.py', 'a', encoding='utf-8') as f:
        f.write(function_code)
    
    # 源文件夹路径
    src_dir = './autoGmail_project/untested functions/%s' % function_name

    # 目标文件夹路径
    dst_dir = './autoGmail_project/tested functions/%s' % function_name
    
    # 移动文件夹
    shutil.move(src_dir, dst_dir)

In [46]:
def show_functions(tested=False, if_print=False):
    """
    打印tested或untested文件夹内全部函数
    """
    current_directory = os.getcwd()
    if tested == False:
        directory = current_directory + '/autoGmail_project/untested functions/'
    else:
        directory = current_directory + '/autoGmail_project/tested functions/'
    files_and_directories = os.listdir(directory)
    # 过滤结果，只保留.py文件和非__pycache__文件夹
    files_and_directories = files_and_directories = [name for name in files_and_directories if (os.path.splitext(name)[1] == '.py' or os.path.isdir(os.path.join(directory, name))) and name != "__pycache__"]
    
    if if_print != False:
        for name in files_and_directories:
            print(name)
    
    return files_and_directories

In [47]:
show_functions(tested=True)

['retrieve_emails', 'get_email_count', 'get_latest_email', 'send_email']

In [48]:
globals()['get_email_count']

<function gptLearning.get_email_count(userId='me')>

In [50]:
get_email_count()

'{"email_count": 201}'

In [52]:
def code_generate(req, few_shot='all', model='gpt-3.5-turbo-16k', g=globals(), detail=0):
    """
    Function calling外部函数自动创建函数，可以根据用户的需求，直接将其翻译为Chat模型可以直接调用的外部函数代码。
    :param req: 必要参数，字符串类型，表示输入的用户需求；
    :param few_shot: 可选参数，默认取值为字符串all，用于描述Few-shot提示示例的选取方案，当输入字符串all时，则代表提取当前外部函数库中全部测试过的函数作为Few-shot；\
    而如果输入的是一个包含了多个函数名称的list，则表示使用这些函数作为Few-shot。
    :param model: 可选参数，表示调用的Chat模型，默认选取gpt-4-0613；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :param detail: 可选参数，默认取值为0，还可以取值为1，表示extract_function_code函数打印新创建的外部函数细节；
    :return：新创建的函数名称。需要注意的是，在函数创建时，该函数也会在当前操作空间被定义，后续可以直接调用；
    """
    
    # 提取提示示例的函数名称
    if few_shot == 'all':
        few_shot_functions_name = show_functions(tested=True)
    elif type(few_shot) == list:
        few_shot_functions_name = few_shot
    # few_shot_functions = [globals()[name] for name in few_shot_functions_name]
    
    # 读取各阶段系统提示
    with open('./autoGmail_project/tested functions/system_messages.json', 'r') as f:
        system_messages = json.load(f)
        
    # 各阶段提示message对象
    few_shot_messages_CM = []
    few_shot_messages_CD = []
    few_shot_messages = []
    
    # 先保存第一条消息，也就是system message
    few_shot_messages_CD += system_messages["system_message_CD"]
    few_shot_messages_CM += system_messages["system_message_CM"]
    few_shot_messages += system_messages["system_message"]

    # 创建不同阶段提示message
    for function_name in few_shot_functions_name:
        with open('./autoGmail_project/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
            msg = json.load(f)
        few_shot_messages_CD += msg["stage1_CD"]
        few_shot_messages_CM += msg["stage1_CM"]
        few_shot_messages += msg['stage2']
        
    # 读取用户需求，作为第一阶段CD环节User content
    new_req_CD_input = req
    few_shot_messages_CD.append({"role": "user", "content": new_req_CD_input})
    
    print('第一阶段CD环节提示创建完毕，正在进行CD提示...')
    
    # 第一阶段CD环节Chat模型调用过程
    response = openai.ChatCompletion.create(
                  model=model,
                  messages=few_shot_messages_CD
                )
    new_req_pi = response.choices[0].message['content']
    
    print('第一阶段CD环节提示完毕')
    
    # 第一阶段CM环节Messages创建
    new_req_CM_input = new_req_CD_input + new_req_pi
    few_shot_messages_CM.append({"role": "user", "content": new_req_CM_input})
    
    print('第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...')
    # 第一阶段CM环节Chat模型调用过程
    response = openai.ChatCompletion.create(
                      model=model,
                      messages=few_shot_messages_CM
                    )
    new_req_description = response.choices[0].message['content']
    
    print('第一阶段CM环节提示完毕')
    
    # 第二阶段Messages创建过程
    few_shot_messages.append({"role": "user", "content": new_req_description})
    
    print('第二阶段提示创建完毕，正在进行第二阶段提示...')
    
    # 第二阶段Chat模型调用过程
    response = openai.ChatCompletion.create(
                  model=model,
                  messages=few_shot_messages
                )
    new_req_function = response.choices[0].message['content']
    
    print('第二阶段提示完毕，准备运行函数并编写提示示例')
    
    # 提取函数并运行，创建函数名称对象，统一都写入untested文件夹内
    function_name = extract_function_code(s=new_req_function, detail=detail, g=g)
    
    print('新函数保存在./autoGmail_project/untested functions/%s/%s_module.py文件中' % (function_name, function_name))
    
    # 创建该函数提示示例
    new_req_messages_CD = [
                          {"role": "user", "content": new_req_CD_input},
                          {"role": "assistant", "content": new_req_pi}
                         ]
    new_req_messages_CM = [
                          {"role": "user", "content": new_req_CM_input},
                          {"role": "assistant", "content":new_req_description}
                         ]
    
    with open('./autoGmail_project/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        new_req_function = f.read()
    
    new_req_messages = [
                       {"role": "user", "content": new_req_description},
                       {"role": "assistant", "content":new_req_function}
                      ] 
    
    new_req_prompt = {
                     "stage1_CD": new_req_messages_CD,
                     "stage1_CM": new_req_messages_CM,
                     "stage2": new_req_messages
                    }   
    
    with open('./autoGmail_project/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(new_req_prompt, f)
        
    print('新函数提示示例保存在./autoGmail_project/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print('done')
    return function_name

In [53]:
few_shot_functions = ['get_latest_email','send_email']
req = "请帮我查下我的邮箱里是否有openai的未读邮件，有的话请解读下这封未读邮件的内容。"

In [54]:
function_name = code_generate(req=req,few_shot=few_shot_functions)

KeyError: 'stage2'